In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-x90m1fwh
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-x90m1fwh
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10733 sha256=25d7336482e00933626da974c4579936e1bcd2169b147b54a7d099795d56a4e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-x56hdm_5/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
# Write your CUDA code to a file
%%writefile hello_cuda.cu
#include <iostream>
__global__ void hello() {
    printf("Hello from CUDA kernel!\n");
}
int main() {
    hello<<<1, 10>>>();
    cudaDeviceSynchronize();
    return 0;
}

Writing hello_cuda.cu


In [4]:
# Компилируем код
!nvcc hello_cuda.cu -o hello_cuda

# Запускаем исполняемый файл
!./hello_cuda

Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!
Hello from CUDA kernel!


In [5]:
%%writefile kernel.cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

#include <string>
#include <iostream>
#include <iomanip>
#include <time.h>
#define BLOCK_SIZE 32

// объявление функций

// функция заполнения матриц рандомными числами
void filling_matrices(float* matrix_1, float* matrix_2, const int matrix_size);

// перемножение матриц на графическом процессоре
__global__ void matrix_multiplication_GPU(float* matrix_1, float* matrix_2, float* multiplication_result, const int matrix_size);

// перемножение матриц на центральном процессоре
float matrix_multiplication_CPU(float* matrix_1, float* matrix_2, float* multiplication_result, const int matrix_size);

// проверка результата перемножения: равно ли перемножение матриц на гпу перемножению матриц на цпу
bool is_equal(float* matrix_1, float* matrix_2, const int matrix_size);


int main(int argc, char* argv[])
{
	// Matrix size is matrix_size * matrix_size.
	const int matrix_size = 512;
	std::cout << "размер матриц: " << matrix_size << std::endl;

	// выделение памяти под матрицы
	float* matrix_1 = new float[matrix_size * matrix_size];
	float* matrix_2 = new float[matrix_size * matrix_size];
	float* cpumultiplication_result = new float[matrix_size * matrix_size];
	float* gpumultiplication_result = new float[matrix_size * matrix_size];

	// заполнение матриц
	filling_matrices(matrix_1, matrix_2, matrix_size);

	// выделение памяти на гпу
	float* cudamatrix_1, * cudamatrix_2, * cudamultiplication_result;
	cudaMalloc((void**)&cudamatrix_1, matrix_size * matrix_size * sizeof(float));
	cudaMalloc((void**)&cudamatrix_2, matrix_size * matrix_size * sizeof(float));
	cudaMalloc((void**)&cudamultiplication_result, matrix_size * matrix_size * sizeof(float));

	// объявление событий для замера времени
	cudaEvent_t start, stop;

	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	// количество нитей и блоков
	dim3 threads(BLOCK_SIZE, BLOCK_SIZE);
	dim3 blocks(matrix_size / threads.x, matrix_size / threads.y);

	// начало записываемого события (замера времени)
	cudaEventRecord(start, 0);

	// копирование матриц на гпу
	cudaMemcpy(cudamatrix_1, matrix_1, matrix_size * matrix_size * sizeof(float), cudaMemcpyHostToDevice);
	cudaMemcpy(cudamatrix_2, matrix_2, matrix_size * matrix_size * sizeof(float), cudaMemcpyHostToDevice);

	// произведение матриц на гпу
	matrix_multiplication_GPU <<< blocks, threads >>> (cudamatrix_1, cudamatrix_2, cudamultiplication_result, matrix_size);

	// копирование результата обратно на хост
	cudaMemcpy(gpumultiplication_result, cudamultiplication_result, matrix_size * matrix_size * sizeof(float), cudaMemcpyDeviceToHost);

	// окончание записи события (замера времени)
	cudaEventRecord(stop, 0);

	cudaEventSynchronize(stop);

	float gpuTime = 0.0f;
	cudaEventElapsedTime(&gpuTime, start, stop);

	std::cout << "\nвремя на GPU " << std::fixed << std::setprecision(2) << gpuTime << " ms\n";

	// произведение матриц на цпу
	float cpuTime = matrix_multiplication_CPU(matrix_1, matrix_2, cpumultiplication_result, matrix_size);
	std::cout << "\nвремя на CPU " << cpuTime << " ms\n";

	std::cout << "\nматрицы одинаковые?: ";
	is_equal(cpumultiplication_result, gpumultiplication_result, matrix_size) ? std::cout << "да" : std::cout << "нет";

	cudaFreeHost(matrix_1);
	cudaFreeHost(matrix_2);
	cudaFreeHost(cpumultiplication_result);
	cudaFreeHost(gpumultiplication_result);
	cudaEventDestroy(start);
	cudaEventDestroy(stop);
	cudaFree(cudamatrix_1);
	cudaFree(cudamatrix_2);
	cudaFree(cudamultiplication_result);

	return 0;
}

void filling_matrices(float* matrix_1, float* matrix_2, const int matrix_size)
{
	srand(time(0));
	float* ptr_1 = matrix_1;
	float* ptr_2 = matrix_2;
	for (int i = 0; i < matrix_size; i++)
	{
		for (int j = 0; j < matrix_size; j++)
		{
			//*(ptr_1 + i * matrix_size + j) = rand() % 1000;
			//*(ptr_2 + i * matrix_size + j) = rand() % 1000;

			*(ptr_1) = rand() % 1000;
			*(ptr_2) = rand() % 1000;
			ptr_1++;
			ptr_2++;
		}
	}
}


__global__ void matrix_multiplication_GPU(float* matrix_1, float* matrix_2, float* multiplication_result, const int matrix_size)
{
	int tx = threadIdx.x;
	int ty = threadIdx.y;

	float sum = 0.0f;
	int indexmatrix_1 = matrix_size * blockDim.y * blockIdx.y + matrix_size * threadIdx.y;
	int indexmatrix_2 = blockDim.x * blockIdx.x + threadIdx.x;
	int indexmultiplication_result = matrix_size * BLOCK_SIZE * blockIdx.y + BLOCK_SIZE * blockIdx.x;

	// вычисление элемента
	for (int k = 0; k < matrix_size; k++)
	{
		sum += matrix_1[indexmatrix_1 + k] * matrix_2[indexmatrix_2 + k * matrix_size];
	}
	multiplication_result[indexmultiplication_result + matrix_size * ty + tx] = sum;
}


float matrix_multiplication_CPU(float* matrix_1, float* matrix_2, float* multiplication_result, const int matrix_size)
{
	clock_t start = clock();
	for (int i = 0; i < matrix_size; i++)
	{
		for (int j = 0; j < matrix_size; j++)
		{
			multiplication_result[i * matrix_size + j] = 0;
			for (int k = 0; k < matrix_size; k++)
			{
				multiplication_result[i * matrix_size + j] += matrix_1[i * matrix_size + k] * matrix_2[k * matrix_size + j];
			}
		}
	}

	clock_t end = clock();
	// перевод времени в млс
	float time = ((float)(end - start) / CLOCKS_PER_SEC) * 1000;

	return time;
}


bool is_equal(float* matrix_1, float* matrix_2, const int matrix_size)
{
	int numBytes = matrix_size * matrix_size * sizeof(float);
	return memcmp(matrix_1, matrix_2, numBytes) == 0; // memcmp из string сравнивает два блока памяти и возвращает 0 если они одинаковые
}

Writing kernel.cu


In [6]:
# Компилируем код
!nvcc kernel.cu -o kernel

# Запускаем исполняемый файл
!./kernel

размер матриц: 512

время на GPU 79.72 ms

время на CPU 1958.30 ms

матрицы одинаковые?: да

128 - 10.96/0.39

256 - 84.24/0.71

512 - 1808.94/2.84

1024 - 11830.79/11.90

2048 - 139993.84/72.23

In [7]:
!pip install altair

In [8]:
import altair as alt
import pandas as pd

# Данные
data = {
    'Size': [128, 256, 512, 1024, 2048],
    'CPU_Time': [10.96, 84.24, 1808.94, 11830.79, 139993.84],
    'GPU_Time': [0.39, 0.71, 2.84, 11.90, 72.23]
}

# Создание DataFrame
df = pd.DataFrame(data)

# Расчет ускорения
df['Speedup'] = df['CPU_Time'] / df['GPU_Time']

# Построение графика
chart = alt.Chart(df).mark_line(point=True).encode(
    x=alt.X('Size:Q', title='Размерность матриц'),
    y=alt.Y('Speedup:Q', title='Ускорение'),
).properties(
    title='Ускорение работы программы на GPU относительно CPU'
)

# Отображение графика
chart

alt.Chart(...)